In [1]:
# test the use of runlength encoding for rainfields rain_rate data 
# a rainfield is identified using stn_id and valid_time
# each stn_id has only one projection 
# TO DO change this when working with forecasts where we get many forecasts for a valid time 
# TO DO at the moment we only read in rain_rate, need to add rainfall accumulation as well 

In [2]:
from pymongo import MongoClient
from netCDF4 import Dataset 
import datetime
import numpy as np 
import collections
import bson
import run_len

In [3]:
# open the nc file and read in the stn_id and valid time
rf3_name="/home/awseed/data/RF3/prcp-crate/2021/20210129/310_20210129_003000.prcp-crate.nc" 
rf3 = Dataset(rf3_name)
stn_id = int(rf3.getncattr("station_id"))
time_stamp = int(rf3['valid_time'][:])
valid_time = datetime.datetime.fromtimestamp(time_stamp) 

In [4]:
# open the database and collections
client = MongoClient()
db = client["test_2"]
proj_col = db["projection"] 
vt_col   = db["valid_time"]
rr_col   = db["rain_rate"]

In [5]:
# check if we have a copy of this field in the database 
count_fields = vt_col.count_documents(filter={"stn_id": stn_id, "valid_time":valid_time}) 

In [6]:
if count_fields == 0:

    # read the valid_time attrubutes from the nc file and insert into database 
    valid_time_attr = rf3.variables['valid_time'].__dict__ 
    valid_time_attr['valid_time'] = valid_time
    valid_time_attr['stn_id'] = stn_id 
    vt_id = vt_col.insert_one(valid_time_attr).inserted_id

    # get the projection details
    proj = {}
    count_docs = proj_col.count_documents(filter={"stn_id": stn_id})
    if count_docs > 0:
        # get the projection from the database 
        proj = proj_col.find_one({"stn_id": stn_id}) 
    else:
        
        # get the projection from the ncfile and write it to the database 
        proj_attr = {}
        proj_attr['grid_mapping_name'] = rf3.variables['proj'].getncattr("grid_mapping_name") 
        standard_parallel = rf3.variables['proj'].getncattr("standard_parallel")
        proj_attr['standard_parallel'] =  standard_parallel.tolist()
        proj_attr['longitude_of_central_meridian'] = rf3.variables['proj'].getncattr("longitude_of_central_meridian")
        proj_attr['latitude_of_projection_origin'] = rf3.variables['proj'].getncattr("latitude_of_projection_origin")
        proj_attr['false_easting'] = rf3.variables['proj'].getncattr("false_easting") 
        proj_attr['false_northing'] = rf3.variables['proj'].getncattr("false_northing") 
        proj_attr['grid_mapping_name'] = rf3.variables['proj'].getncattr("grid_mapping_name") 
        towgs84 = rf3.variables['proj'].getncattr("towgs84")
        proj_attr['towgs84'] = towgs84.tolist()
        
        x_attr = rf3.variables['x'].__dict__ 
        x = rf3['x'][:].tolist()
        x_attr['x'] = x 
        
        y_attr = rf3.variables['y'].__dict__ 
        y = rf3['y'][:].tolist()
        y_attr['y'] = y 
        
        proj['stn_id'] = stn_id
        proj['proj'] = proj_attr 
        proj['y'] = y_attr
        proj['x'] = x_attr

        proj_id = proj_col.insert_one(proj).inserted_id
 
    

In [9]:
# read in metadata for rainfall 
rain_rate_attr = rf3.variables['rain_rate'].__dict__ 
rain_rate_attr['pjn_id'] = stn_id 
rain_rate_attr['valid_time'] = valid_time
rain_rate = rf3['rain_rate'][:] 
rle_rain = run_len.encode(rain_rate, 20)
print (len(rle_rain))
    

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()